In [ ]:
aws rekognition create-collection --collection-id attandence_collection --region us-east-2

In [39]:
import boto3
import io
from PIL import Image
from pathlib import Path
import tempfile

name = 'abcd'
name_with_ext =  name + '.png'
path = 'register/'+ name_with_ext
region_name='us-east-2'
bucket ='attandence-bucket'
collection_id = "attandence_collection"
TableName='attandence_collection'

rekognition = boto3.client('rekognition', region_name = region_name)
dynamodb = boto3.client('dynamodb', region_name = region_name)
s3 = boto3.resource('s3')

file = open(name_with_ext,'rb')
object = s3.Object('attandence-bucket',path)
s3_upload = object.put(Body=file,
                Metadata={'FullName':name}
                )

index_face_response=rekognition.index_faces(CollectionId=collection_id,
                            Image={'S3Object':{'Bucket':bucket,'Name':path}},
                            MaxFaces=1,
                            QualityFilter="AUTO",
                            DetectionAttributes=['ALL'])

faceId = index_face_response['FaceRecords'][0]['Face']['FaceId']

dynamodb_response = dynamodb.put_item(
    TableName = TableName,
    Item={
        'RekognitionId': {'S': faceId},
        'FullName': {'S': name}
        }
    )

faceId


'635305bc-d7f8-4e02-8ba2-4f17c5ef2c3c'

In [36]:
index_face_response

{'FaceRecords': [{'Face': {'FaceId': 'e3ba1690-d133-4e6e-a57e-2b5687b4cf09',
    'BoundingBox': {'Width': 0.5961723327636719,
     'Height': 0.47871923446655273,
     'Left': 0.2804256081581116,
     'Top': 0.3142184019088745},
    'ImageId': 'aa080f88-2c7a-3eda-a9b2-00617c487eb5',
    'Confidence': 99.99990844726562},
   'FaceDetail': {'BoundingBox': {'Width': 0.5961723327636719,
     'Height': 0.47871923446655273,
     'Left': 0.2804256081581116,
     'Top': 0.3142184019088745},
    'AgeRange': {'Low': 23, 'High': 33},
    'Smile': {'Value': False, 'Confidence': 97.58727264404297},
    'Eyeglasses': {'Value': False, 'Confidence': 95.51785278320312},
    'Sunglasses': {'Value': False, 'Confidence': 98.27735137939453},
    'Gender': {'Value': 'Male', 'Confidence': 99.98873138427734},
    'Beard': {'Value': True, 'Confidence': 87.44587707519531},
    'Mustache': {'Value': False, 'Confidence': 62.08765411376953},
    'EyesOpen': {'Value': True, 'Confidence': 97.9516372680664},
    'Mouth

In [40]:
index_face_response

{'FaceRecords': [{'Face': {'FaceId': '635305bc-d7f8-4e02-8ba2-4f17c5ef2c3c',
    'BoundingBox': {'Width': 0.014466222375631332,
     'Height': 0.03450656309723854,
     'Left': 0.7998406887054443,
     'Top': 0.9368987083435059},
    'ImageId': 'cfe56c71-1776-3e8e-969a-c4f14649090f',
    'Confidence': 99.6942138671875},
   'FaceDetail': {'BoundingBox': {'Width': 0.014466222375631332,
     'Height': 0.03450656309723854,
     'Left': 0.7998406887054443,
     'Top': 0.9368987083435059},
    'AgeRange': {'Low': 48, 'High': 56},
    'Smile': {'Value': False, 'Confidence': 92.585205078125},
    'Eyeglasses': {'Value': False, 'Confidence': 97.22028350830078},
    'Sunglasses': {'Value': False, 'Confidence': 98.96420288085938},
    'Gender': {'Value': 'Male', 'Confidence': 99.9949722290039},
    'Beard': {'Value': False, 'Confidence': 80.22366333007812},
    'Mustache': {'Value': False, 'Confidence': 92.09870147705078},
    'EyesOpen': {'Value': True, 'Confidence': 78.05277252197266},
    'Mou

In [57]:
import boto3
import io
from PIL import Image

name = 'abcd'
name_with_ext =  name + '.jpg'
path = 'register/'+ name_with_ext
region_name='us-east-2'
threshold=80

TableName='attandence_collection'
bucket ='attandence-bucket'
collection_id = "attandence_collection"

rekognition = boto3.client('rekognition', region_name = region_name)
dynamodb = boto3.client('dynamodb', region_name = region_name)
s3 = boto3.resource('s3')

response = rekognition.search_faces_by_image(
    Image={
        "S3Object": {
            "Bucket": bucket,
            "Name": path,
        }
    },
    CollectionId=collection_id,
    FaceMatchThreshold=threshold)

faceId = response['FaceMatches'][0]['Face']['FaceId']
similarity = response['FaceMatches'][0]['Similarity']

dynamodb_response = dynamodb.get_item(
    TableName=TableName,  
    Key={'RekognitionId':{'S': faceId}})


RekognitionId = dynamodb_response['Item']['RekognitionId']['S']
FullName = dynamodb_response['Item']['FullName']['S']

RekognitionId,FullName

InvalidS3ObjectException: An error occurred (InvalidS3ObjectException) when calling the SearchFacesByImage operation: Unable to get object metadata from S3. Check object key, region and/or access permissions.

In [59]:
s3 = boto3.resource('s3')

# Get list of objects for indexing
images=[('abcd.png','Albert Einstein')]

# Iterate through list to upload objects to S3   
for image in images:
    file = open(image[0],'rb')
    object = s3.Object('attandence-bucket','index/'+ image[0])
    ret = object.put(Body=file,
                    Metadata={'FullName':image[1]}
                    )

In [79]:
import boto3
import io
from PIL import Image

rekognition = boto3.client('rekognition', region_name='us-east-2')
dynamodb = boto3.client('dynamodb', region_name='us-east-2')
    
image = Image.open("bb.jpg")
stream = io.BytesIO()
image.save(stream,format="JPEG")
image_binary = stream.getvalue()


response = rekognition.search_faces_by_image(
        CollectionId='attandence_collection',
        Image={'Bytes':image_binary}                                       
        )
for match in response['FaceMatches']:
    print (match['Face']['FaceId'],match['Face']['Confidence'])
        
    face = dynamodb.get_item(
        TableName='attandence_collection',  
        Key={'RekognitionId': {'S': match['Face']['FaceId']}}
        )
    
    if 'Item' in face:
        print (face['Item']['FullName']['S'])
    else:
        print ('no match found in person lookup')

19808c2c-3233-44f1-8330-c50c030fff78 99.99979400634766
no match found in person lookup


In [30]:
import boto3

BUCKET = "attandence-bucket"
KEY_SOURCE = "index/bb.jpg"
KEY_TARGET = "index/01.jpg"

def compare_faces(bucket, key, bucket_target, key_target, threshold=80, region="us-east-2"):
    rekognition = boto3.client("rekognition", region)
    response = rekognition.compare_faces(
        SourceImage={
            "S3Object": {
                "Bucket": bucket,
                "Name": key,
            }
        },
        TargetImage={
            "S3Object": {
                "Bucket": bucket_target,
                "Name": key_target,
            }
        },
           SimilarityThreshold=threshold,
    )
    print (response)
    
    return(response['SourceImageFace'], response['FaceMatches'])


source_face, matches = compare_faces(BUCKET, KEY_SOURCE, BUCKET, KEY_TARGET)

# the main source face
print ("Source Face ({Confidence}%)".format(**source_face))

# one match for each target face
for match in matches:
    print ("Target Face ({Confidence}%)".format(**match['Face']))
    print ("  Similarity : {}%".format(match['Similarity']))


InvalidS3ObjectException: An error occurred (InvalidS3ObjectException) when calling the CompareFaces operation: Unable to get object metadata from S3. Check object key, region and/or access permissions.

In [77]:
import boto3

BUCKET = "attandence-bucket"
KEY = "index/bb.jpg"
COLLECTION = "attandence_collection"

def search_faces_by_image(bucket, key, collection_id, threshold=80, region="us-east-2"):
    rekognition = boto3.client("rekognition", region)
    response = rekognition.search_faces_by_image(
        Image={
            "S3Object": {
                "Bucket": bucket,
                "Name": key,
            }
        },
        CollectionId=collection_id,
        FaceMatchThreshold=threshold,
    )
    return (response['FaceMatches'])

for record in search_faces_by_image(BUCKET, KEY, COLLECTION):
    face = record['Face']
    print ("Matched Face ({}%)".format(record['Similarity']))
    print ("  FaceId : {}".format(face['FaceId']))
#     print ("  ImageId : {}".format(face['ExternalImageId']))


Matched Face (100.0%)
  FaceId : 19808c2c-3233-44f1-8330-c50c030fff78


In [74]:
client=boto3.client('rekognition')

dynamodb = boto3.client('dynamodb')
s3 = boto3.client('s3')


def add_faces_to_collection(bucket,photo,collection_id):

    response = dynamodb.put_item(
        TableName=bucket,
        Item={
            'RekognitionId': {'S': faceId},
            'FullName': {'S': fullName}
            }
        ) 

    response=client.index_faces(CollectionId=collection_id,
                                Image={'S3Object':{'Bucket':bucket,'Name':photo}},
                                MaxFaces=1,
                                QualityFilter="AUTO",
                                DetectionAttributes=['ALL'])

    print ('Results for ' + photo) 
    print('Faces indexed:')
    for faceRecord in response['FaceRecords']:
         print('  Face ID: ' + faceRecord['Face']['FaceId'])
         print('  Location: {}'.format(faceRecord['Face']['BoundingBox']))

    print('Faces not indexed:')
    for unindexedFace in response['UnindexedFaces']:
        print(' Location: {}'.format(unindexedFace['FaceDetail']['BoundingBox']))
        print(' Reasons:')
        for reason in unindexedFace['Reasons']:
            print('   ' + reason)
    return len(response['FaceRecords'])


bucket='attandence-bucket'
collection_id='attandence_collection'
photo="index/aa.jpg"


indexed_faces_count=add_faces_to_collection(bucket, photo, collection_id)
print("Faces indexed count: " + str(indexed_faces_count))



Results for index/aa.jpg
Faces indexed:
  Face ID: 44a6198c-919f-419e-990c-884dabafd440
  Location: {'Width': 0.5961723327636719, 'Height': 0.47871923446655273, 'Left': 0.2804256081581116, 'Top': 0.3142184019088745}
Faces not indexed:
Faces indexed count: 1


In [31]:
import boto3

def list_faces_in_collection(collection_id):


    maxResults=2
    faces_count=0
    tokens=True

    client=boto3.client('rekognition')
    response=client.list_faces(CollectionId=collection_id,
                               MaxResults=maxResults)

    print('Faces in collection ' + collection_id)
 
    while tokens:

        faces=response['Faces']

        for face in faces:
            print (face)
            faces_count+=1
        if 'NextToken' in response:
            nextToken=response['NextToken']
            response=client.list_faces(CollectionId=collection_id,
                                       NextToken=nextToken,MaxResults=maxResults)
        else:
            tokens=False
    return faces_count   
def main():

    collection_id='attandence_collection'
    
    faces_count=list_faces_in_collection(collection_id)
    print("faces count: " + str(faces_count))
if __name__ == "__main__":
    main()

Faces in collection attandence_collection
{'FaceId': '635305bc-d7f8-4e02-8ba2-4f17c5ef2c3c', 'BoundingBox': {'Width': 0.014466200023889542, 'Height': 0.034506600350141525, 'Left': 0.7998409867286682, 'Top': 0.9368990063667297}, 'ImageId': 'cfe56c71-1776-3e8e-969a-c4f14649090f', 'Confidence': 99.69419860839844}
{'FaceId': '68166d1b-feef-4f40-bb78-68c9f0b67dcb', 'BoundingBox': {'Width': 0.3412570059299469, 'Height': 0.32137998938560486, 'Left': 0.3092130124568939, 'Top': 0.4611220061779022}, 'ImageId': 'cb8e9b69-00a2-3e6e-9ca9-14b0a3cb50f7', 'Confidence': 99.99979400634766}
{'FaceId': 'e3ba1690-d133-4e6e-a57e-2b5687b4cf09', 'BoundingBox': {'Width': 0.5961719751358032, 'Height': 0.47871899604797363, 'Left': 0.2804259955883026, 'Top': 0.31421801447868347}, 'ImageId': 'aa080f88-2c7a-3eda-a9b2-00617c487eb5', 'Confidence': 99.9999008178711}
faces count: 3


In [1]:
#Copyright 2018 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#PDX-License-Identifier: MIT-0 (For details, see https://github.com/awsdocs/amazon-rekognition-developer-guide/blob/master/LICENSE-SAMPLECODE.)

import boto3
from botocore.exceptions import ClientError
from os import environ

def delete_collection(collection_id):


    print('Attempting to delete collection ' + collection_id)
    client=boto3.client('rekognition')
    status_code=0
    try:
        response=client.delete_collection(CollectionId=collection_id)
        status_code=response['StatusCode']
        
    except ClientError as e:
        if e.response['Error']['Code'] == 'ResourceNotFoundException':
            print ('The collection ' + collection_id + ' was not found ')
        else:
            print ('Error other than Not Found occurred: ' + e.response['Error']['Message'])
        status_code=e.response['ResponseMetadata']['HTTPStatusCode']
    return(status_code)


def main():
    collection_id='attandence_collection'
    status_code=delete_collection(collection_id)
    print('Status code: ' + str(status_code))

if __name__ == "__main__":
    main()  


Attempting to delete collection attandence_collection
Status code: 200


In [61]:
import boto3


BUCKET = "attandence-bucket"
KEY = "index/abcd.png"
FEATURES_BLACKLIST = ("Landmarks", "Emotions", "Pose", "Quality", "BoundingBox", "Confidence")

def detect_faces(bucket, key, attributes=['ALL'], region="us-east-2"):
    rekognition = boto3.client("rekognition", region)
    response = rekognition.detect_faces(
        Image={
            "S3Object": {
                "Bucket": bucket,
                "Name": key,
            }
        },
        Attributes=attributes,
    )
    return response['FaceDetails']

for face in detect_faces(BUCKET, KEY):
    print ("Face ({Confidence}%)".format(**face))
    # emotions
    for emotion in face['Emotions']:
        print ("  {Type} : {Confidence}%".format(**emotion))
    # quality
    for quality, value in face['Quality'].iteritems():
        print ("  {quality} : {value}".format(quality=quality, value=value))
    # facial features
    for feature, data in face.iteritems():
        if feature not in FEATURES_BLACKLIST:
            print ("  {feature}({data[Value]}) : {data[Confidence]}%".format(feature=feature, data=data))


Face (99.6942138671875%)
  SURPRISED : 81.62865447998047%
  CALM : 8.259970664978027%
  ANGRY : 4.233240604400635%
  FEAR : 2.26208233833313%
  CONFUSED : 1.805971622467041%
  DISGUSTED : 0.7482640743255615%
  SAD : 0.5447291731834412%
  HAPPY : 0.5170879364013672%


AttributeError: 'dict' object has no attribute 'iteritems'

In [4]:
a='<div class="contaienr"><div class="col-lg-4 mb-5"> '+'<div class="practicing">'+'<div class="practicing-inner">'+'<div class="wrap-icon">'+'<span class="flaticon-museum"></span>'+'</div>'+'<h3>Bankruptcy Law</h3>'+'<p>Lorem ipsum dolor sit amet, consectetur adipisicing elit. Vero, laboriosam!</p>'+'</div>'+'</div>'+'</div>'+'<div class="col-lg-4 mb-5">'+'<div class="practicing">'+'<div class="practicing-inner">'+'<div class="wrap-icon">'+'<span class="flaticon-shield"></span>'+'</div>'+'<h3>Business Law</h3>'+'<p>Lorem ipsum dolor sit amet, consectetur adipisicing elit. Vero, laboriosam!</p>'+'</div>'+'</div>'+'</div>'+'<div class="col-lg-4 mb-5">'+'<div class="practicing">'+'<div class="practicing-inner">'+'<div class="wrap-icon">'+'<span class="flaticon-help"></span>'+'</div>'+'<h3>Civil Rights Law</h3>'+'<p>Lorem ipsum dolor sit amet, consectetur adipisicing elit. Vero, laboriosam!</p>'+'</div>'+'</div>'+'</div>'+'<div class="col-lg-4 mb-5">'+'<div class="practicing">'+'<div class="practicing-inner">'+'<div class="wrap-icon">'+'<span class="flaticon-revolver"></span>'+'</div>'+'<h3>Criminal Law</h3>'+'<p>Lorem ipsum dolor sit amet, consectetur adipisicing elit. Vero, laboriosam!</p>'+'</div>'+'</div>'+'</div>'+'<div class="col-lg-4 mb-5">'+'<div class="practicing">'+'<div class="practicing-inner">'+'<div class="wrap-icon">'+'<span class="flaticon-luggage"></span>'+'</div>'+'<h3>Immigration Law</h3>'+'<p>Lorem ipsum dolor sit amet, consectetur adipisicing elit. Vero, laboriosam!</p>'+'</div>'+'</div>'+'</div>'+'<div class="col-lg-4 mb-5">'+'<div class="practicing">'+'<div class="practicing-inner">'+'<div class="wrap-icon">'+'<span class="flaticon-house"></span>'+'</div>'+'<h3>Family Law</h3>'+'<p>Lorem ipsum dolor sit amet, consectetur adipisicing elit. Vero, laboriosam!</p>'+'</div>'+'</div>'+'</div></div>'

In [5]:
a

'<div class="contaienr"><div class="col-lg-4 mb-5"> <div class="practicing"><div class="practicing-inner"><div class="wrap-icon"><span class="flaticon-museum"></span></div><h3>Bankruptcy Law</h3><p>Lorem ipsum dolor sit amet, consectetur adipisicing elit. Vero, laboriosam!</p></div></div></div><div class="col-lg-4 mb-5"><div class="practicing"><div class="practicing-inner"><div class="wrap-icon"><span class="flaticon-shield"></span></div><h3>Business Law</h3><p>Lorem ipsum dolor sit amet, consectetur adipisicing elit. Vero, laboriosam!</p></div></div></div><div class="col-lg-4 mb-5"><div class="practicing"><div class="practicing-inner"><div class="wrap-icon"><span class="flaticon-help"></span></div><h3>Civil Rights Law</h3><p>Lorem ipsum dolor sit amet, consectetur adipisicing elit. Vero, laboriosam!</p></div></div></div><div class="col-lg-4 mb-5"><div class="practicing"><div class="practicing-inner"><div class="wrap-icon"><span class="flaticon-revolver"></span></div><h3>Criminal Law</